### ADD Brand List of Shopinng Mall

In [1]:
import sys

sys.path.append("../../")

In [2]:
# in Jupyter
import re
import sys
import os
from random import randint
from typing import List, Dict, Any, Callable

import asyncio 
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, Page, TimeoutError as PlaywrightTimeoutError 



pw = await async_playwright().start()
browser = await pw.firefox.launch(headless=False)
page = await browser.new_page()


page.set_default_timeout(15000)

async def _load_page( page: Page, url):
    """페이지 로드"""
    await asyncio.sleep(1)
    await page.goto(url)
    await page.wait_for_load_state(state="networkidle")
    await page.wait_for_timeout(1000)
    return page


In [3]:
await _load_page(page, "https://kream.co.kr/login")

<Page url='https://kream.co.kr/login'>

In [6]:
async def save_cookies(page):
    # Get cookies from the current page
    cookies = await page.context.cookies()

    # Save cookies to a file or database
    # In this example, cookies are saved to a file named 'cookies.json'
    with open('../../router/dev/kream/cookie/cookies.json', 'w') as file:
        file.write(str(cookies))


In [4]:
async def load_cookies(page):
    # Load cookies from a file or database
    # In this example, cookies are loaded from a file named 'cookies.json'
    
    with open('../../router/dev/kream/cookie/cookies.json', 'r') as file:
        cookies = eval(file.read())

    # Set cookies in the current page
    await page.context.add_cookies(cookies)

await load_cookies(page)
await page.goto("https://kream.co.kr")

<Response url='https://kream.co.kr/' request=<Request url='https://kream.co.kr/' method='GET'>>

In [ ]:
from datetime import date, datetime

async def _scrap_trading_volume_from(
    page: Page, target_date: date, kream_id: int, **_kwargs
):
    volume_btn = "a[class='btn outlinegrey full medium']"
    await page.wait_for_selector(volume_btn, timeout=5000)

    trading_volume_button = await page.query_selector(volume_btn)

    assert trading_volume_button, "trading_volume_button not found"
    await trading_volume_button.click()

    price_btn = "div[class='price_body']"
    try : 
        await page.wait_for_selector(price_btn, timeout=5000)
    except Exception as e:
        print(f"{kream_id} : 체결내역 더보기가 없는 제품으로 추정")
        return []


    modal = await page.query_selector(price_btn)
    assert modal, "modal not found"

    # scroll until target_date
    i = 0
    start = 0
    end = 1
    volumes = []
    while start < end and i <= 20:
        i += 1
        start = end
        await asyncio.sleep(1)
        end = await page.evaluate("Array.from(document.querySelectorAll('.list_txt.is_active')).length")
        date_str= await page.evaluate(f"Array.from(document.querySelectorAll('.list_txt.is_active')).slice(-1)[0].innerText")
        last_date = datetime.strptime(date_str, "%y/%m/%d").date()

        print(f"kream_id:{kream_id},last_date : {last_date}, target_date : {target_date}, end : {end}")

        if target_date < last_date:
            scroll_eval = (
                "(async () => { document.querySelector('.price_body').scrollBy(0, 5000); })()"
            )
            await page.evaluate(scroll_eval)
        else:
            break

    if len(volumes) == 0:
        return []
    
await _scrap_trading_volume_from(page, date(2023, 10, 24), 10)

In [9]:
x = {'x':'asd'}

x.update({'y':'asd'})

x

{'x': 'asd', 'y': 'asd'}